In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import socket

# Load your trained YOLOv8 model
model = YOLO('best.pt')  # Replace with 'yolov8n.pt' if needed

# Load corner points calibration data
corner_points = np.load('corner_points.npz')['corner_points']
tl, tr, bl, br = corner_points[:4]

# Load drop points data
drop_points = np.load('drop_points.npz', allow_pickle=True)
drop_points_dict = drop_points['drop_points'].item()

# Define class names (must match your trained model)
class_names = ['AA', 'D', '9V']

# Robot IP and Port
ROBOT_IP = "192.168.150.2"
ROBOT_PORT = 2001

# Initialize camera (adjust camera_index as needed)
camera_index = 0  # Example: change this to match your external camera index
cap = cv2.VideoCapture(camera_index)

if not cap.isOpened():
    print(f"Error: Could not open camera with index {camera_index}")
    exit()

# Establish TCP/IP connection to the robot
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((ROBOT_IP, ROBOT_PORT))

# Define the home position
home_position = (370, 205, 380)

# Function to map detected position to normalized grid coordinates
def map_to_grid(x, y, tl, tr, bl, br):
    v1 = np.array([tr[0] - tl[0], tr[1] - tl[1]])
    v2 = np.array([bl[0] - tl[0], bl[1] - tl[1]])
    vp = np.array([x - tl[0], y - tl[1]])

    u = np.dot(vp, v1) / np.dot(v1, v1)
    v = np.dot(vp, v2) / np.dot(v2, v2)

    return u, v

# Function to convert grid coordinates to robot coordinates
def grid_to_robot(u, v, tl, tr, bl, br):
    x_robot = tl[0] + u * (tr[0] - tl[0]) + v * (bl[0] - tl[0])
    y_robot = tl[1] + u * (tr[1] - tl[1]) + v * (bl[1] - tl[1])
    z_robot = 162  # Initial Z coordinate
    return x_robot, y_robot, z_robot

# Function to calculate X2, Y2, Z with adjustment based on battery type
def calculate_X2_Y2_Z(x1, y1, z1, battery_type):
    X2 = 0.71 * y1 + 351.32
    Y2 = 0.5381 * x1 - 0.1054 * y1 + 78

    # Adjust Z based on battery type
    if battery_type == 'D':
        Z2 = z1 + 32.5
    elif battery_type == '9V':
        Z2 = z1 + 16.7
    elif battery_type == 'AA':
        Z2 = z1 + 13.7
    else:
        Z2 = z1

    return int(X2), int(Y2), int(Z2)

# Function to get the drop point based on battery type
def get_drop_point(battery_type):
    if battery_type == 'AA':
        return drop_points_dict['AA']
    elif battery_type == '9V':
        return drop_points_dict['9V']
    elif battery_type == 'D':
        return drop_points_dict['D']
    else:
        raise ValueError("Unknown battery type")

# Function to send movement commands to the robot
def send_robot_command(x, y, z):
    command = f"{x} {y} {z}\r\n"
    sock.send(command.encode())
    print(f"Sent command: {command}")

# Function to control the electromagnet (ON/OFF)
def control_electromagnet(state):
    # Send 'PICK' to activate electromagnet (ON) or 'DROP' to deactivate (OFF)
    if state == "ON":
        command = "PICK 8\r\n"  # Assuming 'PICK 8' activates pin 8
    elif state == "OFF":
        command = "DROP 8\r\n"  # Assuming 'DROP 8' deactivates pin 8
    else:
        raise ValueError("Invalid state for electromagnet. Use 'ON' or 'OFF'.")
    
    sock.send(command.encode())
    print(f"Sent electromagnet command: {command}")

try:
    while True:
        ret, frame = cap.read()  # Capture frame-by-frame

        if not ret:
            print("Error: Failed to capture image")
            break

        # Display the frame without detection
        cv2.imshow('Battery Detection', frame)

        # Wait for spacebar ('space') press to trigger detection
        if cv2.waitKey(1) & 0xFF == ord(' '):
            # Move robot to the home position first
            send_robot_command(*home_position)

            # Perform inference when spacebar is pressed
            results = model(frame)[0]  # Get the first result

            # Iterate through the detected objects
            for result in results.boxes.data:
                x1, y1, x2, y2, confidence, class_id = result.tolist()

                # Calculate center coordinates of the bounding box
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2

                # Map the detected position to the grid coordinates
                u, v = map_to_grid(x_center, y_center, tl, tr, bl, br)

                # Convert grid coordinates to robot coordinates
                x_robot, y_robot, z_robot = grid_to_robot(u, v, tl, tr, bl, br)

                # Calculate new X2, Y2, Z based on the formula provided and battery type
                battery_type = class_names[int(class_id)]
                new_X2, new_Y2, new_Z = calculate_X2_Y2_Z(x_center, y_center, z_robot, battery_type)

                # Move robot to the detected battery position
                send_robot_command(new_X2, new_Y2, new_Z)

                # Turn ON the electromagnet (pick up the battery)
                control_electromagnet("ON")

                # Move back to the home position
                send_robot_command(*home_position)

                # Get the drop point and send the robot to drop position
                drop_point = get_drop_point(battery_type)
                send_robot_command(*drop_point)

                # Turn OFF the electromagnet (drop the battery)
                control_electromagnet("OFF")

                # Move back to the home position
                send_robot_command(*home_position)

            # Display the frame with detections
            cv2.imshow('Battery Detection', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

except KeyboardInterrupt:
    print("Detection stopped by user.")
finally:
    # Release the capture and close all windows
    cap.release()
    cv2.destroyAllWindows()
    sock.close()


C:\Users\mfaiz\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\nn\tasks.py:781: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, m

Sent command: 370 205 380

Robot response: OK


0: 480x640 1 D type, 1 9v, 270.2ms
Speed: 18.5ms preprocess, 270.2ms inference, 10.3ms postprocess per image at shape (1, 3, 480, 640)
Sent command: 448 279 194

Robot response: OK

Sent electromagnet command: PICK 8

Detection stopped by user.
